# Creating batch to test partial feedback setting

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import sys
sys.path.append('../')

In [3]:
import source.players as pl
import source.parsers as parsers
import source.game as game
import source.player as player
import source.environment as environment
import source.errors as errors
import source.runner as runner
import source.util as util
import source.players.attackers as attackers
import source.players.base_defenders as base_defenders
import source.players.defenders as defenders
import source.players.holmes as holmes
import source.players.sad as sad
import source.players.b2bw2w as b2bw2w
import source.players.belief_max as bm
import source.players.dmd as dmd
import source.players.bayesian_approximator as ba

import concurrent.futures
import pkgutil
import logging
import sys
from source.util import *
import numpy as np
import pandas as pd
from importlib import *
import time
import os
import re
from copy import deepcopy, copy
import datetime
from contextlib import redirect_stdout
from random import shuffle

In [4]:
# run once

lvl = logging.DEBUG
logger = logging.getLogger("source")
logger.setLevel(lvl)
#ch = logging.StreamHandler(sys.stdout)
ch = logging.FileHandler("status", mode='w')
ch.setLevel(lvl)
#formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
formatter = logging.Formatter('%(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

In [5]:
lvl = logging.WARNING
logger.setLevel(lvl)
ch.setLevel(lvl)

In [6]:
# if you change something in the files, don't forget to reload
reload(player)
reload(base_defenders)
reload(attackers)
reload(defenders)
reload(errors)
reload(environment)
reload(runner)
reload(parsers)
reload(game)
reload(holmes)
reload(sad)
reload(b2bw2w)
reload(util)
reload(bm)
reload(dmd)

<module 'source.players.dmd' from '..\\source\\players\\dmd.py'>

### Creating configurations to save into a csv file

In [16]:
time_horizon = 100
T = 20
#targets = util.gen_tar_with_len(T)
targets = [1 for i in range(T)]
values = tuple((v, v) for v in targets)
#val_ls = [values[i][0] for i in range(T)]
#print(val_ls)
feedback_prob = [1 for i in range(T)]
observabilities = [1 for i in range(T)]
feedback_type = "full"
distributions = []
#for i in range(5):
#    distributions.append(tuple(util.gen_distr(T)))
#print(distributions)
distributions.append(tuple([1/T for i in range(T)]))
#chosen_distr = tuple(util.gen_distr(T))
#chosen_distr = tuple([1/T for i in range(T)])
chosen_distr = tuple([0.031] + [0.051 for i in range(19)])
#print("chosen = ", chosen_distr)
#len(chosen_distr)

In [17]:
g = game.GameWithObservabilities(values, time_horizon, observabilities, feedback_prob, feedback_type = "mab")
g.attackers = [1]
g.defenders = [0]
d = [bm.FB(g, 0)]
prof = []
for distr in distributions:
    prof.append(attackers.StochasticAttacker(g, 1, 1, *distr))
prof.append(attackers.StochasticAttacker(g, 1, 1, *chosen_distr))
prof.append(attackers.StackelbergAttacker(g, 1, 1))
#for i in range(5):
#    prof.append(attackers.SUQR(g,1))
#chosen_a_prof = (attackers.FrequentistUnknownStochasticAttacker(g, 1, 1))
#chosen_a_prof = attackers.SUQR(g,1)
#prof.append(attackers.FrequentistUnknownStochasticAttacker(g, 1, 1))
#a = [deepcopy(chosen_a_prof)]
#prof.append(chosen_a_prof)
a = [attackers.StochasticAttacker(g, 1, 1, *chosen_distr)]
#a = [attackers.StackelbergAttacker(g, 1, 1)]
#shuffle(prof)
#print("Real attacker is: ", a[0])
g.set_players(d, a, prof)

In [18]:
def print_header(targets, profiles):
    return "Name,T," + ",".join(str(i) for i in range(len(targets))) + ",Defender,Attacker," + ",".join(["Profile" for x in range(len(profiles))]) + "," + ",".join(["Obs" + str(x) for x in range(len(targets))]) + "," + ",".join(["Feed_prob" + str(x) for x in range(len(targets))]) + ",Feed_type" + "\n" 

In [19]:
def print_row(targets, time_horizon, d, a, p, obs, fprob, ftype):
    return ",".join([str(i) for i in ([d+"_vs_"+a] + [time_horizon] + targets + [d] + [a] + [x for x in p] + [o for o in obs] + [fp for fp in fprob] + [ftype]) ])+"\n"

In [20]:
(print_header(targets, prof))

'Name,T,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,Defender,Attacker,Profile,Profile,Profile,Obs0,Obs1,Obs2,Obs3,Obs4,Obs5,Obs6,Obs7,Obs8,Obs9,Obs10,Obs11,Obs12,Obs13,Obs14,Obs15,Obs16,Obs17,Obs18,Obs19,Feed_prob0,Feed_prob1,Feed_prob2,Feed_prob3,Feed_prob4,Feed_prob5,Feed_prob6,Feed_prob7,Feed_prob8,Feed_prob9,Feed_prob10,Feed_prob11,Feed_prob12,Feed_prob13,Feed_prob14,Feed_prob15,Feed_prob16,Feed_prob17,Feed_prob18,Feed_prob19,Feed_type\n'

In [21]:
defender =["FB1"]

In [22]:
profile_strings = []
for p in prof:
    profile_strings.append(util.print_adv(p))

In [23]:
experiments_folder = "../NewExperiments"
exp_subfolder = "/Full_vs_MAB"
batch_name = "extreme_case_6"
batch_file = batch_name + ".csv"
batch_path = experiments_folder + exp_subfolder + "/" + batch_file

### Generating the csv and checking everything is fine

In [24]:
with open(batch_path, "w+") as f:
    f.write(print_header(targets, prof))
    f.write(print_row(targets, time_horizon,defender[0],util.print_adv(a[0]),profile_strings,observabilities,feedback_prob,feedback_type))

In [18]:
pd.read_csv(batch_path)

,Name,T,0,1,2,3,4,5,Defender,Attacker,...,Obs3,Obs4,Obs5,Feed_prob0,Feed_prob1,Feed_prob2,Feed_prob3,Feed_prob4,Feed_prob5,Feed_type
0,FB1_vs_sto1-0.4984984984984984-0.1481481481481...,100,0.711,0.61,0.791,0.63,0.544,0.664,FB1,sto1-0.4984984984984984-0.1481481481481481-0.1...,...,1,1,1,1,1,1,1,1,1,full
